In [260]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

url = "https://api.hypixel.net/skyblock/auctions"

r = requests.get(url).json()
total_pages = int(r['totalPages'])
total_pages
df3 = pd.DataFrame()
def fetch_auction_house(page_number:int) -> pd.DataFrame:
        response = requests.get(f'{url}?page={page_number}')
        auctions_data = response.json()['auctions']
        df = pd.DataFrame(auctions_data)
        df.drop(columns=['auctioneer','profile_id','coop', 'extra','item_lore','tier','item_bytes','claimed_bidders','item_uuid','bids'],inplace=True)
        return df

def clean_dataframe(df:pd.DataFrame) -> pd.DataFrame:
        df['start'] = pd.to_datetime(df['start'], unit='ms')
        df['end'] = pd.to_datetime(df['end'], unit='ms')
        df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
        return df



for i in range(total_pages):
   df3 = pd.concat( [df3, fetch_auction_house(i)])

df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,df4c2f41d16d472aaa6897b4a15152f8,2024-07-14 04:02:31.898,2024-07-14 10:02:31.898,Training Dummy,misc,294000,False,0,2024-07-14 04:02:31.898,True
1,dc95516691e04c5f84038da295c142a6,2024-07-14 04:02:31.666,2024-07-14 10:02:31.666,Experience Artifact,accessories,910000,False,0,2024-07-14 04:02:31.666,True
2,ac3d277a985b40d6b4050c9cb9e9de79,2024-07-14 04:02:31.588,2024-07-16 04:02:31.588,[Lvl 52] Mooshroom Cow,misc,8500000,False,0,2024-07-14 04:02:31.588,True
3,3d01f47184614567a7416203a2c3c5b5,2024-07-14 04:02:31.519,2024-07-16 04:02:31.519,Jaded Sorrow Boots,armor,13700000,False,0,2024-07-14 04:02:31.519,True
4,f83dca74698e45c7b8e49a17aefd5406,2024-07-14 04:02:31.300,2024-07-14 10:02:31.300,Velvet Top Hat,armor,84999,False,0,2024-07-14 04:02:31.300,True
...,...,...,...,...,...,...,...,...,...,...
574,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
575,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
576,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
577,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [261]:
# Using threading improved avg runtime from 25 - 30seconds to 3 - 4 (Thanks ChatGPT for the help)

url = "https://api.hypixel.net/skyblock/auctions"
r = requests.get(url).json()
total_pages = int(r['totalPages'])


# For loops through the total_number of pages on the market place, requesting JSON for each page
# Each page contains a max of 1000 Items
def fetch_auction_house(page_number: int) -> pd.DataFrame:
    response = requests.get(f'{url}?page={page_number}')
    auctions_data = response.json()['auctions']
    df = pd.DataFrame(auctions_data)
    return df
# Clean DataFrame
# Remove unnescessary code
# Change all timestamps from UNIX to yyyy/mm/dd hour/min/sec/ms format
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['auctioneer', 'profile_id', 'coop', 'extra', 'item_lore', 'tier', 'item_bytes', 'claimed_bidders', 'item_uuid', 'bids'], inplace=True)
    df['start'] = pd.to_datetime(df['start'], unit='ms')
    df['end'] = pd.to_datetime(df['end'], unit='ms')
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
    return df


#ChatGPT helped me with this one, ill need someone to explain this to me
# Use ThreadPoolExecutor to fetch pages concurrently
def fetch_all_pages_concurrently(total_pages: int) -> pd.DataFrame:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(fetch_auction_house, i) for i in range(total_pages)]
        result = pd.concat([future.result() for future in futures], ignore_index=True)
    return result

df3 = fetch_all_pages_concurrently(total_pages)
df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,45ca1dad012a43f08c95a6fa9e97b28b,2024-07-14 03:02:45.566,2024-07-14 04:05:31.888,Vaccine Ring,accessories,100,False,552365,2024-07-14 04:03:31.845,False
1,852c3d0bcd4f452893f17dd2624a4f5a,2024-07-14 04:03:31.401,2024-07-14 10:03:31.401,Kat Flower,blocks,699999,False,0,2024-07-14 04:03:31.401,True
2,7040de32fa0f42e9b3e889afa810c2b4,2024-07-14 04:03:31.305,2024-07-28 04:03:31.305,[Lvl 1] Turtle,misc,25500000,False,0,2024-07-14 04:03:31.305,True
3,b6307cab56854dac8b4cb78510bee47b,2024-07-13 04:06:37.514,2024-07-14 04:06:37.514,Legendary Raider Axe,weapon,99999,False,340000,2024-07-14 04:03:30.308,False
4,d199a4da32df4af3bd5aa90a5dd2cecb,2024-07-14 04:03:29.627,2024-07-14 10:03:29.627,[Lvl 1] Slug,misc,2500000,False,0,2024-07-14 04:03:29.627,True
...,...,...,...,...,...,...,...,...,...,...
58575,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
58576,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
58577,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
58578,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [262]:
def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return "Item not with specified Parameters"
    return filtered_df

find_item('Portal Dye', 79000000)

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
355,/viewauction 852a02bdb21e47838bdce94c1e7b5fcf,2024-07-14 04:01:38.854,2024-07-14 10:01:38.854,Portal Dye,misc,76500000,False,0,2024-07-14 04:01:38.854,True
385,/viewauction f9eb03ce41744901bd0b4856513dc268,2024-07-14 04:01:27.739,2024-07-14 10:01:27.739,Portal Dye,misc,76500000,False,0,2024-07-14 04:01:27.739,True
671,/viewauction 375f23ad32bb4c708458d7db971021ab,2024-07-14 03:59:46.253,2024-07-28 03:59:46.253,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:46.253,True
647,/viewauction 685a33170ec94aea99fb10e836c1da40,2024-07-14 03:59:55.304,2024-07-28 03:59:55.304,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:55.304,True
695,/viewauction 6082db19dc4044febd1cae95678b5705,2024-07-14 03:59:36.800,2024-07-28 03:59:36.800,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:36.800,True
1852,/viewauction 4629c31edc974339aff69d14675eaca2,2024-07-14 03:52:11.537,2024-07-14 09:52:11.537,Portal Dye,misc,77999000,False,0,2024-07-14 03:52:11.537,True
166,/viewauction 2dba915d46fd4a81ba15d06ebd4f63d3,2024-07-14 04:02:37.503,2024-07-16 04:02:37.503,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:37.503,True
148,/viewauction fef4a9abccd94495b5fff45d2250b914,2024-07-14 04:02:43.288,2024-07-16 04:02:43.288,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:43.288,True
129,/viewauction ed54a7201784408bbf3c5978e18e95ff,2024-07-14 04:02:49.881,2024-07-16 04:02:49.881,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:49.881,True
107,/viewauction 768f2955a9364daba1a3eeec6f08006e,2024-07-14 04:02:56.285,2024-07-16 04:02:56.285,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:56.285,True
